In [1]:
import pandas as pd
from pandas.io import gbq

In [2]:
table_df = gbq.read_gbq("SELECT * FROM `umg-comm-tech-dev.fixed_playlists_data.playlists_list`",
                       project_id = "umg-comm-tech-dev", dialect = 'standard')
table_df.head()

,artists,genre,playlist_owner,release_date,release_title,repertoire_pools,status,streaming_services,take_down_date,territory,upc
0,None,None,None,2016-11-23 00:00:00+00:00,Christmas,None,None,None,NaT,None,00602557103601
1,None,None,None,2016-12-09 00:00:00+00:00,Christmas Blues,None,None,None,NaT,None,00602557342697
2,None,None,None,2016-12-09 00:00:00+00:00,Christmas Jazz,None,None,None,NaT,None,00602557342703
3,None,None,None,2017-01-27 00:00:00+00:00,Crowd Pleasers,None,None,None,NaT,None,00602557414387
4,None,None,None,2017-02-03 00:00:00+00:00,Let Me Love You Old School,None,None,None,NaT,None,00602557437638


In [3]:
table_df.describe(include='all')

,artists,genre,playlist_owner,release_date,release_title,repertoire_pools,status,streaming_services,take_down_date,territory,upc
count,633,633,633,811,811,633,633,573,60,573,811
unique,5,38,44,134,728,15,7,3,27,15,811
top,Various artists,Jazz,Jamie Ells,2019-06-21 00:00:00+00:00,Bossa Nova Hits,Catalogue,DELIVERED,Spotify;Deezer;Napster;7digital;Amazon;Tidal;Y...,2018-10-09 00:00:00+00:00,AR;AT;AU;BO;BR;CA;CH;CL;CO;CR;DE;DK;EC;ES;FI;F...,00602577016813
freq,628,177,123,50,5,341,575,546,13,305,1
first,NaN,NaN,NaN,2016-11-23 00:00:00+00:00,NaN,NaN,NaN,NaN,2018-02-20 00:00:00+00:00,NaN,NaN
last,NaN,NaN,NaN,2019-06-28 00:00:00+00:00,NaN,NaN,NaN,NaN,2035-07-02 00:00:00+00:00,NaN,NaN


In [4]:
dtype_dict = {
    'upc':str
}

col_names = ['upc','release_title','artists', 'genre', 'status', 'release_date', 'take_down_date', 'playlist_owner', 
       'repertoire_pools', 'streaming_services', 'territory']

In [5]:
new_data = pd.read_csv('/Users/meshchd/Downloads/compilations-2019-06-28.csv', index_col=None, dtype=dtype_dict,
                      header=0, names=col_names)
new_data.head()

,upc,release_title,artists,genre,status,release_date,take_down_date,playlist_owner,repertoire_pools,streaming_services,territory
0,NaN,Jazz Standards: The 1920s,Various artists,Jazz,DRAFT,2018-05-11,NaN,Kayla Schmandt,Catalogue,Spotify;Deezer;Napster;7digital;Amazon,AR;AT;AU;BO;BR;CA;CH;CL;CO;CR;DE;DK;DO;EC;ES;F...
1,00600406836304,Blues For Breakfast,Various artists,Blues,REOPENED,2018-05-04,NaN,James Burnett,Catalogue,Spotify;Deezer;Napster;7digital;Amazon,AR;AT;AU;BO;BR;CA;CH;CL;CO;CR;DE;DK;DO;EC;ES;F...
2,NaN,Punk Hits,Various artists,Punk,CLOSED,2018-05-07,NaN,Kayla Schmandt,Catalogue,Spotify;Deezer;Napster;7digital;Amazon,AR;AT;AU;BO;BR;CA;CH;CL;CO;CR;DE;DK;DO;EC;ES;F...
3,00600406836311,Midnight Jazz,Various artists,Jazz,DELIVERED,2018-05-11,NaN,Kayla Schmandt,Classical / Jazz,Spotify;Deezer;Napster;7digital;Amazon,AR;AT;AU;BO;BR;CA;CH;CL;CO;CR;DE;DK;DO;EC;ES;F...
4,00600406836366,Smooth Jazz,Various artists,Jazz,DELIVERED,2018-06-08,2018-10-10,Kayla Schmandt,Classical / Jazz,Spotify;Deezer;Napster;7digital;Amazon,AR;AT;AU;BO;BR;CA;CH;CL;CO;CR;DE;DK;DO;EC;ES;F...


In [6]:
truly_new = new_data[(new_data.upc.isnull()==False)&(new_data.upc.isin(table_df.upc.unique())==False)]

truly_new['release_date'] = pd.to_datetime(truly_new.release_date)
truly_new['take_down_date'] = pd.to_datetime(truly_new.take_down_date)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [7]:
all_data = pd.concat([table_df,truly_new], ignore_index=True, sort=True)
all_data['release_date'] = pd.to_datetime(all_data.release_date, utc=True)
all_data['take_down_date'] = pd.to_datetime(all_data.take_down_date, utc=True)
all_data.upc.describe()

count                863
unique               863
top       00602577016813
freq                   1
Name: upc, dtype: object

In [8]:
all_data.to_gbq(destination_table="fixed_playlists_data.playlists_list",
                       project_id = "umg-comm-tech-dev", if_exists = 'replace')

1it [00:04,  4.70s/it]


In [9]:
len(truly_new.upc.unique())

52

In [10]:
all_data.shape

(863, 11)